# ResDNN = ResNet + DNN
Residual Blocks

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

plt.style.use('seaborn')
sns.set(font_scale=2.5)
import missingno as msno

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

from sklearn.preprocessing import RobustScaler

# Check Dataset

In [ ]:
path = '../input/tabular-playground-series-nov-2021/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')

In [ ]:
print(train.shape)
train.head()

In [ ]:
train.describe()

In [ ]:
test.head()

In [ ]:
test.describe()

## Null Data Check

Result : No Null Data

In [ ]:
for col in train.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (train[col].isnull().sum() / train[col].shape[0]))
    print(msg)

In [ ]:
msno.matrix(df=train.iloc[:, :], figsize=(8, 8), color=(0.8, 0.5, 0.2))

## Target Label Check

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))

train['target'].value_counts().plot.pie(explode=[0, 0.1], autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('Pie plot - target')
ax[0].set_ylabel('')
sns.countplot('target', data=train, ax=ax[1])
ax[1].set_title('Count plot - target')

plt.show()

# EDA

Check Distribution of Features

Nothing special

no cumulative features

In [ ]:
fig, ax = plt.subplots(10, 10, figsize=(50, 50))
for i in range(0, 10):
  for j in range(0, 10):
    col = f"f{10*i + j}"
    print(f"...{col}", end=" ")
    sns.kdeplot(train[train['target'] == 1][col], ax=ax[i][j])
    sns.kdeplot(train[train['target'] == 0][col], ax=ax[i][j])
  print("")
plt.legend(['target == 1', 'target == 0'])
plt.show()

In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize=(500, 500))
plt.title('Correlation of Features', y=1.05, size=15)
sns.heatmap(train.corr(), linewidths=0.1, vmax=1.0,
           square=True, cmap=colormap, linecolor='white', annot=True, annot_kws={"size": 16})

Result: Nothing Special except the distibution ~> need to scale ... I will chooose Robust Scaler because it is need to be deal with outliers.

Feature Names are also no evidence for looking for 

# Feature Engineering

Result: Just Do It

In [ ]:
X_train = train.drop(['id', 'target'], axis=1)
y_train = train['target']

X_test = test.drop(['id'], axis=1)

In [ ]:
X_train

In [ ]:
X_test

# Normalizing ~ Robust Scaler

Scaling : Robust Scaler for dealing with outliers

In [ ]:
# Robust Scaler
RS = RobustScaler().fit(X_train)
X_train = RS.transform(X_train)
X_test = RS.transform(X_test)

# Build Model

In [ ]:
import tensorflow as tf

from sklearn.model_selection import StratifiedKFold

In [ ]:
class ResBlock(tf.keras.layers.Layer):
    def __init__(self, n, dropout_rate=0.1):

        super(ResBlock, self).__init__()

        self.dense1 = tf.keras.layers.Dense(1024*n, activation='relu', kernel_regularizer='l2')
        self.dense2 = tf.keras.layers.Dense(512*n, activation='relu', kernel_regularizer='l2')

        self.dense_res = tf.keras.layers.Dense(512*n, activation='relu')

        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    def call(self, inputs, training):
        
        y1 = self.dense1(inputs)
        y1 = self.dropout1(y1)

        y2 = self.dense2(y1)
        y2 = self.dropout2(y2)

        y_res = self.dense_res(inputs)

        return y2 + y_res

In [ ]:
feat_dim = 1024
num_blocks=0
n = 1
dropout_rate = 0.1

def build_model():
    inputs = tf.keras.Input(shape=(100,))

    x = tf.keras.layers.Dense(feat_dim)(inputs)

    for k in range(num_blocks):
        resBlock = ResBlock(n, dropout_rate)
        x = resBlock(x)

    for k in range(1, 9, 2):
        resBlock = ResBlock(1/(2**k), dropout_rate)
        x = resBlock(x)

    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
# Thanks to Kaggle Notebook "TensorFlow Transformer - [0.112]"
import math

LR_START = 1e-6
LR_MAX = 6e-4
LR_MIN = 1e-6
LR_RAMPUP_EPOCHS = 0
LR_SUSTAIN_EPOCHS = 0
EPOCHS = 420
STEPS = [60,120,240]


def lrfn(epoch):
    if epoch<STEPS[0]:
        epoch2 = epoch
        EPOCHS2 = STEPS[0]
    elif epoch<STEPS[0]+STEPS[1]:
        epoch2 = epoch-STEPS[0]
        EPOCHS2 = STEPS[1]
    elif epoch<STEPS[0]+STEPS[1]+STEPS[2]:
        epoch2 = epoch-STEPS[0]-STEPS[1]
        EPOCHS2 = STEPS[2]
    
    if epoch2 < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch2 + LR_START
    elif epoch2 < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        decay_total_epochs = EPOCHS2 - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS - 1
        decay_epoch_index = epoch2 - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS
        phase = math.pi * decay_epoch_index / decay_total_epochs
        cosine_decay = 0.5 * (1 + math.cos(phase))
        lr = (LR_MAX - LR_MIN) * cosine_decay + LR_MIN
    return lr

lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

rng = [i for i in range(EPOCHS)]
lr_y = [lrfn(x) for x in rng]
plt.figure(figsize=(10, 4))
plt.plot(rng, lr_y, '-o')
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}". \
          format(lr_y[0], max(lr_y), lr_y[-1]))
plt.xlabel('Epoch',size=14)
plt.ylabel('Learning Rate',size=14)
plt.show()
del lr_callback

# Train Model & Prediction

I will tune num_blocks and num_fold

In [ ]:
X_test.shape[0]

In [ ]:
EPOCH = 60
BATCH_SIZE = 2**14

num_fold = 10 # 20

skf = StratifiedKFold(num_fold, shuffle=True, random_state=21) 

prediction = np.zeros((X_test.shape[0], 1)) # for Ensenble

i = 0
for train_index, valid_index in skf.split(X_train, y_train):
    i += 1
    
    print('\n', '='*15, '>>>', f'Fold {i}', '<<<', '='*15)

    model = build_model()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics = ['AUC'])
    
    save_path = './'
    checkpoint_folderpath = save_path + f"weights/{i}/"
    checkpoint_filepath = save_path + f"weights/{i}/weights"
    if os.path.isdir(checkpoint_folderpath):
        print(f"Loading Fold #{i} Weights")
        model.load_weights(checkpoint_filepath)

    #lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)
    sv = tf.keras.callbacks.ModelCheckpoint(
            checkpoint_filepath, monitor='val_auc', verbose=1, save_best_only=True,
            save_weights_only=True, mode='max', save_freq='epoch', options=None)
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=5)
    
    history = model.fit(X_train[train_index], y_train[train_index], verbose=1,
                              validation_data=(X_train[valid_index], y_train[valid_index]),
                              epochs=EPOCH, batch_size=BATCH_SIZE, callbacks=[sv, early_stop]) # lr_callback, 
    # Prediction 
    print(f'Fold {i} ~ Predicting Test...')
    prediction = np.concatenate((prediction, model.predict(X_test, batch_size=BATCH_SIZE, verbose=1)), axis=1)
    
    del model
    del optimizer
    # del lr_callback
    del sv
    # early_stop

In [ ]:
prediction.shape

In [ ]:
prediction = prediction[:, 1:]
prediction.shape

# OOF Ensemble

In [ ]:
# Already Done on Training Cell
"""
prediction = np.zeros((X_test.shape[0], 1))
for i in range(1, 10+1):
    print('\n', '='*15, '>>>', f'Fold {i}', '<<<', '='*15)

    model = build_model()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics = ['AUC'])
    
    save_path = './'
    checkpoint_folderpath = save_path + f"weights/{i}/"
    checkpoint_filepath = save_path + f"weights/{i}/weights"
    if os.path.isdir(checkpoint_folderpath):
        print(f"Loading Fold #{i} Weights")
        model.load_weights(checkpoint_filepath)
    else:
        print("=>ERROR<="*15)
        break
    
    y_pred = model.predict(X_test, verbose=1, batch_size=BATCH_SIZE)
    prediction += y_pred

prediction /= 10
"""

In [ ]:
prediction_mean = np.sum(prediction, axis=-1)
prediction_median = np.median(prediction, axis=-1)

In [ ]:
prediction_mean.shape

In [ ]:
prediction_median.shape

# Create Submission File

In [ ]:
submission = pd.read_csv(path+'sample_submission.csv')

In [ ]:
submission["target"] = prediction_mean
submission.to_csv(f'submission_mean.csv', index=False)

In [ ]:
submission["target"] = prediction_median
submission.to_csv('submission_median.csv', index=False)

# median -> submit
submission.to_csv('submission.csv', index=False)